In [32]:
#Importing the Libraries
import os   #used for listing directories in a Folder, or Items in a File
import torch  # Pytorch Neural Network Package
import pandas as pd    # To import csv Files
import numpy as np      # To operate on Numpy Arrays
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader   # To create a custom DataSet in Pytorch
from torchvision import transforms, utils     # To operate on Datasets such as Normilazation
import pydicom    # To deal Operate on DICOM Images
import csv
import random
from skimage.transform import resize

In [75]:
import torch
from torch.utils import data
import torch.optim as optim
from torchvision.models import resnet50

import torch.nn as nn
import torch.nn.functional as F

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
# empty dictionary
pneumonia_locations = {}
# load table
with open(os.path.join('./rsna-pneumonia-detection-challenge/stage_2_train_labels.csv'), mode='r') as infile:
    # open reader
    reader = csv.reader(infile)
    # skip header
    next(reader, None)
    # loop through rows
    for rows in reader:
        # retrieve information
        filename = rows[0]
        location = rows[1:5]
        pneumonia = rows[5]
        # if row contains pneumonia add label to dictionary
        # which contains a list of pneumonia locations per filename
        if pneumonia == '1':
            # convert string to float to int
            location = [int(float(i)) for i in location]
            # save pneumonia location in dictionary
            if filename in pneumonia_locations:
                pneumonia_locations[filename].append(location)
            else:
                pneumonia_locations[filename] = [location]

In [35]:
# load and shuffle filenames
folder = './rsna-pneumonia-detection-challenge/stage_2_train_images'
filenames = os.listdir(folder)
random.shuffle(filenames)
# split into train and validation filenames
n_valid_samples = 2560
train_filenames = filenames[n_valid_samples:]
valid_filenames = filenames[:n_valid_samples]
print('n train samples', len(train_filenames))
print('n valid samples', len(valid_filenames))
n_train_samples = len(filenames) - n_valid_samples

n train samples 24124
n valid samples 2560


In [64]:
#Let us a create a class for the Training Dataset

class PnomniaData(Dataset):
    
    # Lets create a PnemoniaDataset Object Creator
    def __init__(self,filenames,folder,image_size):
        self.filenames = filenames
        self.folder = folder
        self.image_size = image_size
        
    # Let us Find the Size of the DataSet
    def __len__(self):
        return len(self.filenames)
    
    #Lets Create a method to Get an Item of the Dataset
    def __getitem__(self, idx):  #Idx is the index of the wanted Item of The Dataset
        filename = self.filenames[idx]
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        msk = np.zeros(img.shape)
        filename = filename.split('.')[0]
        if filename in pneumonia_locations:
            for location in pneumonia_locations[filename]:
                x, y, w, h = location
                msk[y:y+h, x:x+w] = 1
                    
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        msk = resize(msk, (self.image_size, self.image_size), mode='reflect') > 0.5
        img = np.expand_dims(img, -1)
        msk = np.expand_dims(msk, -1)
        imgs = torch.tensor(img).to(device).type(torch.FloatTensor)
        msks = torch.tensor(msk).to(device).type(torch.FloatTensor)
        imgs = imgs.permute(2, 0, 1)
        msks = msks.permute(2, 0, 1)
        return imgs,msks
        

    
 

In [65]:
folder = './rsna-pneumonia-detection-challenge/stage_2_train_images'
IMAGE_SIZE = 320
trainingDataset = PnomniaData(train_filenames,folder,image_size=IMAGE_SIZE)

#Print some Data
for i in range(len(trainingDataset)):
    imgs, msks = trainingDataset[i]
    

    print(i, imgs.sum(),msks.sum())

    if i == 10:
        break

0 tensor(45786.3984) tensor(0.)
1 tensor(43910.3008) tensor(0.)
2 tensor(52731.8867) tensor(0.)
3 tensor(50823.3438) tensor(0.)
4 tensor(49635.4062) tensor(0.)
5 tensor(46528.0586) tensor(0.)
6 tensor(37432.8086) tensor(0.)
7 tensor(50669.6055) tensor(0.)
8 tensor(43862.7344) tensor(0.)
9 tensor(43525.2266) tensor(15193.)
10 tensor(48493.3672) tensor(0.)


In [66]:
trainloader = DataLoader(trainingDataset , batch_size = 4,
                        shuffle = True ,num_workers = 4)
#testloader = DataLoader(testingDataset , batch_size = 4,
                       # shuffle = True ,num_workers = 4)


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Neti(nn.Module):
    def __init__(self):
        super(Neti, self).__init__()
        self.conv1 = nn.Conv2d(1,20,10)
        self.conv2 = nn.Conv2d(20,40,10)
        self.conv3 = nn.Conv2d(40,64,10)
        self.fc1 = nn.Linear(1600, 120)
        self.fc2 = nn.Linear(120,100)
        self.fc3 = nn.Linear(100,5)
        
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(5,5))
        x = F.max_pool2d(F.relu(self.conv2(x)),5)
        x = F.max_pool2d(F.relu(self.conv3(x)),5)
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_fetures = 1
        for s in size:
            num_fetures *=s
        return num_fetures
    
    

In [76]:
class conv_block(nn.Module):
    """
    Define the [convolution - batch normalization - activation] block 
    """

    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True,
                 bn_momentum=0.9, alpha_leaky=0.03):
        super(conv_block, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size,
                              stride=stride, padding=padding, bias=bias)
        self.bn = nn.BatchNorm2d(out_channels, eps=1e-05, momentum=bn_momentum)
        self.activ = nn.LeakyReLU(negative_slope=alpha_leaky)

    def forward(self, x):
        return self.activ(self.bn(self.conv(x)))
    

class conv_t_block(nn.Module):
    """
    Define the [convolution_transpose - batch normalization - activation] block 
    """

    def __init__(self, in_channels, out_channels, output_size=None, kernel_size=3, bias=True,
                 bn_momentum=0.9, alpha_leaky=0.03):
        super(conv_t_block, self).__init__()
        self.conv_t = nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=2, padding=1, 
                                         bias=bias)
        self.bn = nn.BatchNorm2d(out_channels, eps=1e-05, momentum=bn_momentum)
        self.activ = nn.LeakyReLU(negative_slope=alpha_leaky)

    def forward(self, x, output_size):
        return self.activ(self.bn(self.conv_t(x, output_size=output_size)))   

In [81]:
class Net(nn.Module):
    '''
    Custom ResNet module
    '''

    def __init__(self):
        super(Net, self).__init__()
        
        self.down_1 = nn.Sequential(conv_block(in_channels=1, out_channels=64), conv_block(in_channels=64, out_channels=64))
        self.down_2 = nn.Sequential(conv_block(in_channels=64, out_channels=128), conv_block(in_channels=128, out_channels=128))
        self.down_3 = nn.Sequential(conv_block(in_channels=128, out_channels=256), conv_block(in_channels=256, out_channels=256))
        self.down_4 = nn.Sequential(conv_block(in_channels=256, out_channels=512), conv_block(in_channels=512, out_channels=512))
        self.down_5 = nn.Sequential(conv_block(in_channels=512, out_channels=512), conv_block(in_channels=512, out_channels=512))

        self.middle = nn.Sequential(conv_block(in_channels=512, out_channels=512), conv_block(in_channels=512, out_channels=512))
        self.middle_t = conv_t_block(in_channels=512, out_channels=256)

        self.up_5 = nn.Sequential(conv_block(in_channels=768, out_channels=512), conv_block(in_channels=512, out_channels=512))
        self.up_5_t = conv_t_block(in_channels=512, out_channels=256)
        self.up_4 = nn.Sequential(conv_block(in_channels=768, out_channels=512), conv_block(in_channels=512, out_channels=512))
        self.up_4_t = conv_t_block(in_channels=512, out_channels=128)
        self.up_3 = nn.Sequential(conv_block(in_channels=384, out_channels=256), conv_block(in_channels=256, out_channels=256))
        self.up_3_t = conv_t_block(in_channels=256, out_channels=64)
        self.up_2 = nn.Sequential(conv_block(in_channels=192, out_channels=128), conv_block(in_channels=128, out_channels=128))
        self.up_2_t = conv_t_block(in_channels=128, out_channels=32)
        self.up_1 = nn.Sequential(conv_block(in_channels=96, out_channels=64), conv_block(in_channels=64, out_channels=1))
        
    def forward(self, x):
        down1 = self.down_1(x)
        out = F.max_pool2d(down1, kernel_size=2, stride=2)

        down2 = self.down_2(out)
        out = F.max_pool2d(down2, kernel_size=2, stride=2)

        down3 = self.down_3(out)
        out = F.max_pool2d(down3, kernel_size=2, stride=2)

        down4 = self.down_4(out)
        out = F.max_pool2d(down4, kernel_size=2, stride=2)

        down5 = self.down_5(out)
        out = F.max_pool2d(down5, kernel_size=2, stride=2)

        out = self.middle(out)
        out = self.middle_t(out, output_size=down5.size())

        out = torch.cat([down5, out], 1)
        out = self.up_5(out)
        out = self.up_5_t(out, output_size=down4.size())

        out = torch.cat([down4, out], 1)
        out = self.up_4(out)
        out = self.up_4_t(out, output_size=down3.size())
        
        out = torch.cat([down3, out], 1)
        out = self.up_3(out)
        out = self.up_3_t(out, output_size=down2.size())
        
        out = torch.cat([down2, out], 1)
        out = self.up_2(out)
        out = self.up_2_t(out, output_size=down1.size())
        
        out = torch.cat([down1, out], 1)
        out = self.up_1(out)
        return out

In [82]:
def iou_loss(y_true, y_pred):
    '''
    Intersection-Over-Union Loss
    '''
    y_true = torch.reshape(y_true, [-1])
    y_pred = torch.reshape(y_pred, [-1])
    intersection = (y_true * y_pred).sum()
    score = (intersection + 1.) / (y_true.sum() + y_pred.sum() - intersection + 1.)
    return 1 - score

def iou_bce_loss(y_true, y_pred):
    '''
    Main loss function using:
        Binary Cross Entropy +
        Intersection-Over-Union Loss
    '''
    return 0.5 * F.binary_cross_entropy_with_logits(y_true, y_pred) + 0.5 * iou_loss(y_true, y_pred)


def mean_iou(y_true, y_pred, device):
    '''
    Mean-Intersection-Over-Union
    '''
    y_pred = torch.round(y_pred)
    intersect = (y_true * y_pred).sum(axis=[1, 2, 3])
    union = y_true.sum(axis=[1, 2, 3]) + y_pred.sum(axis=[1, 2, 3])
    smooth = torch.ones(intersect.shape).to(device)
    return ((intersect + smooth) / (union - intersect + smooth)).sum()

In [83]:
net = Net().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [84]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        x, y = data

        optimizer.zero_grad()

        out = net(x)
        loss = iou_bce_loss(out, y)
        loss.backward()

        optimizer.step()

        iou = mean_iou(out, y, device)

        print("Batch_ndx:{0:5d}, Loss:{1:2.4f}, Mean-Intersection-Over-Union:{2:2.4f}"
                .format(i, loss, iou))

Batch_ndx:    0, Loss:0.9451, Mean-Intersection-Over-Union:0.0001
Batch_ndx:    1, Loss:0.8934, Mean-Intersection-Over-Union:0.0875
Batch_ndx:    2, Loss:0.8772, Mean-Intersection-Over-Union:0.1162
Batch_ndx:    3, Loss:0.8961, Mean-Intersection-Over-Union:0.0002
Batch_ndx:    4, Loss:0.8937, Mean-Intersection-Over-Union:0.0002
Batch_ndx:    5, Loss:0.8862, Mean-Intersection-Over-Union:0.0003
Batch_ndx:    6, Loss:0.8707, Mean-Intersection-Over-Union:0.0948


KeyboardInterrupt: 

In [86]:
out.shape

torch.Size([4, 1, 320, 320])